In [1]:
!pip install sentence-transformers
!pip install transformers datasets tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
import zipfile
import ast
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AdamW
from transformers import TrainingArguments, Trainer
import random
from tqdm import tqdm


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Paths
zip_file_path = '/content/SemEval_Task7.zip'

In [4]:
!ls -l {zip_file_path}

-rw-r--r-- 1 root root 37555441 Nov 27 12:13 /content/SemEval_Task7.zip


In [5]:
# Unzip the dataset
dataset_path = '/content/datasets'
os.makedirs(dataset_path, exist_ok=True)
!chmod 777 {dataset_path}
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

In [6]:
# Define file paths inside the extracted directory
posts_path = os.path.join(dataset_path, 'SemEval_Task7/posts.csv')
fact_checks_path = os.path.join(dataset_path, 'SemEval_Task7/fact_checks.csv')
pairs_path = os.path.join(dataset_path, 'SemEval_Task7/pairs.csv')

# Now, load the data
df_posts = pd.read_csv(posts_path).fillna('').set_index('post_id')
df_fact_checks = pd.read_csv(fact_checks_path).fillna('').set_index('fact_check_id')
df_pairs = pd.read_csv(pairs_path)

In [7]:
# Example of parsing columns if necessary
parse_col = lambda s: ast.literal_eval(s.replace('\n', '\\n')) if s else s

# Apply the parsing logic to specific columns if required
for col in ['instances', 'ocr', 'verdicts', 'text']:
    df_posts[col] = df_posts[col].apply(parse_col)

for col in ['claim', 'instances', 'title']:
    df_fact_checks[col] = df_fact_checks[col].apply(parse_col)

In [8]:
# df_fact_check_post_mapping = pd.read_csv(fact_check_post_mapping_path)
print("Df_Posts:",len(df_posts))
# df_fact_checks.head()
print("mapping",len(df_pairs))

Df_Posts: 24431
mapping 25743


In [9]:
# Load tasks.json
with open('/content/datasets/SemEval_Task7/tasks.json', 'r') as f:
    tasks_data = json.load(f)

# Extract fact checks and posts for monolingual and crosslingual tasks
monolingual = tasks_data['monolingual']['eng']
crosslingual = tasks_data['crosslingual']

# Restrict the posts and fact checks for the train and dev sets
monolingual_train_posts = monolingual['posts_train']
monolingual_dev_posts = monolingual['posts_dev']
monolingual_fact_checks = monolingual['fact_checks']

# crosslingual_train_posts = crosslingual['posts_train']
# crosslingual_dev_posts = crosslingual['posts_dev']
# crosslingual_fact_checks = crosslingual['fact_checks']


In [10]:
# monolingual

In [11]:
# Filter the datasets accordingly
df_monolingual_train_posts = df_posts.loc[monolingual_train_posts]

df_monolingual_fact_checks = df_fact_checks.loc[monolingual_fact_checks]

# df_crosslingual_train_posts = df_posts.loc[crosslingual_train_posts]
# df_crosslingual_fact_checks = df_fact_checks.loc[crosslingual_fact_checks]

# print(df_monolingual_train_posts['text'])
df_monolingual_train_posts.head()
# df_monolingual_fact_checks.head()


instances  \
post_id                                             
2        [(1610052141.0, fb), (1610072448.0, fb)]   
5                            [(1595754861.0, fb)]   
13                           [(1473438408.0, fb)]   
14                           [(1601665028.0, fb)]   
15       [(1603642996.0, fb), (1603523078.0, fb)]   

                                                       ocr  \
post_id                                                      
2        [("Actually, he's a damn sight better than any...   
5        [("Cigarette smoking does not cause cancer." -...   
13       [("Environmentalists" Say Fracking is Evil 53-...   
14       [("Environmentalists" Say Fracking is Evil SAP...   
15       [("Environmentalists" Say Fracking is Evil Thi...   

                           verdicts text  
post_id                                   
2                 [Missing context]       
5               [False information]       
13       [Partly false information]       
14       [Partly false information]       
15       [Partly false information]

In [12]:
# Sample 100 random posts from the larger dataset (assuming 30,000 posts in actual data)
# Tried for segementation first but now its all english post
sampled_posts = df_monolingual_train_posts
len(sampled_posts)

# Get the post_ids of the selected 100 posts
sampled_post_ids = sampled_posts.index.tolist()

# Filter the mapping dataframe to only include rows corresponding to the 100 selected posts
filtered_mapping = df_pairs[df_pairs['post_id'].isin(sampled_post_ids)]

# Get the fact_check_ids that correspond to the selected posts
sampled_fact_ids = filtered_mapping['fact_check_id'].unique().tolist()

# Filter the facts dataframe to only include the selected fact_check_ids (assuming 250,000 facts in actual data)
filtered_facts = df_monolingual_fact_checks[df_monolingual_fact_checks.index.isin(sampled_fact_ids)]

In [13]:
df_monolingual_train_posts.head()

instances  \
post_id                                             
2        [(1610052141.0, fb), (1610072448.0, fb)]   
5                            [(1595754861.0, fb)]   
13                           [(1473438408.0, fb)]   
14                           [(1601665028.0, fb)]   
15       [(1603642996.0, fb), (1603523078.0, fb)]   

                                                       ocr  \
post_id                                                      
2        [("Actually, he's a damn sight better than any...   
5        [("Cigarette smoking does not cause cancer." -...   
13       [("Environmentalists" Say Fracking is Evil 53-...   
14       [("Environmentalists" Say Fracking is Evil SAP...   
15       [("Environmentalists" Say Fracking is Evil Thi...   

                           verdicts text  
post_id                                   
2                 [Missing context]       
5               [False information]       
13       [Partly false information]       
14       [Partly false information]       
15       [Partly false information]

In [14]:
len(filtered_facts)

3347

In [15]:
filtered_facts.head()

claim  \
fact_check_id                                                      
33             ("$4 trillion jobs plan" unnecessary because 2...   
433            ("Absent from the news — Dominion lost their l...   
1298           ("Congress Just Gave Itself A 21% Raise As Ame...   
1501           ("Dominion LOST their law suits against Rudy G...   
1941           ("Florida has had 119 hurricanes since 1850,” ...   

                                                       instances  \
fact_check_id                                                      
33             [(1620152520.0, https://factcheck.afp.com/post...   
433            [(1622332680.0, https://www.politifact.com/fac...   
1298           [(1647388740.0, https://www.factcheck.org/2022...   
1501           [(1622419080.0, https://www.factcheck.org/2021...   
1941           [(1631059080.0, https://www.politifact.com/fac...   

                                                           title  
fact_check_id                                                     
33             (Posts on Biden jobs plan falsely claim 2020 u...  
433            (Dominion’s defamation lawsuit against Giulian...  
1298           (Spending Bill Includes Pay Raise for Staffers...  
1501           (Dominion’s Defamation Lawsuits Are Still Acti...  
1941           (Experts say climate change can make hurricane...

In [16]:
filtered_mapping.head()

post_id  fact_check_id
0     2228             33
1     2228          23568
2     2228         194577
3     2229             33
4     2229          23568

In [17]:
# Ensure that there are no NaN or float values in the 'claim' column
df_monolingual_fact_checks['claim'] = df_monolingual_fact_checks['claim'].fillna('')  # Replace NaNs with an empty string
df_monolingual_fact_checks['claim'] = df_monolingual_fact_checks['claim'].astype(str)  # Convert all claims to strings
filtered_facts['claim'] = filtered_facts['claim'].fillna('')  # Replace NaNs with an empty string
filtered_facts['claim'] = filtered_facts['claim'].astype(str)  # Convert all claims to strings

df_mdp = df_posts.loc[monolingual_dev_posts]
df_mdp['text'] = df_mdp['text'].fillna('')  # Replace NaNs with an empty string
df_mdp['text'] = df_mdp['text'].astype(str)  # Convert all claims to strings


# Ensure that the 'text' column only contains strings
df_monolingual_train_posts['text'] = df_monolingual_train_posts['text'].fillna('')  # Fill NaN values with empty strings
df_monolingual_train_posts['text'] = df_monolingual_train_posts['text'].astype(str)  # Convert all entries to strings

# Similarly, for the development set
df_mdp['text'] = df_mdp['text'].fillna('')
df_mdp['text'] = df_mdp['text'].astype(str)

# Check if any non-string or problematic data exists
print(df_monolingual_train_posts['text'].apply(lambda x: type(x)).value_counts())
print(df_mdp['text'].apply(lambda x: type(x)).value_counts())

text
<class 'str'>    4351
Name: count, dtype: int64
text
<class 'str'>    478
Name: count, dtype: int64


<ipython-input-17-5b1b6e1eb2eb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_facts['claim'] = filtered_facts['claim'].fillna('')  # Replace NaNs with an empty string
<ipython-input-17-5b1b6e1eb2eb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_facts['claim'] = filtered_facts['claim'].astype(str)  # Convert all claims to strings


In [18]:
def extract_first_ocr(ocr):
    """Extract the first text from the 'ocr' list of tuples if it exists."""
    if isinstance(ocr, list) and len(ocr) > 0 and isinstance(ocr[0], tuple) and len(ocr[0]) > 0:
        return str(ocr[0][0])  # Extract the first element of the first tuple
    return ""  # Return an empty string if no valid ocr exists

# Combine 'ocr' and 'text' into the 'text' column
df_monolingual_train_posts['text'] = df_monolingual_train_posts.apply(
    lambda row: f"{extract_first_ocr(row['ocr'])} {row['text']}".strip(), axis=1
)

df_mdp['text'] = df_mdp.apply(
    lambda row: f"{extract_first_ocr(row['ocr'])} {row['text']}".strip(), axis=1
)

# Verify the results
print(df_monolingual_train_posts[['ocr', 'text']].head())
print(df_mdp[['ocr', 'text']].head())


                                                       ocr  \
post_id                                                      
2        [("Actually, he's a damn sight better than any...   
5        [("Cigarette smoking does not cause cancer." -...   
13       [("Environmentalists" Say Fracking is Evil 53-...   
14       [("Environmentalists" Say Fracking is Evil SAP...   
15       [("Environmentalists" Say Fracking is Evil Thi...   

                                                      text  
post_id                                                     
2        "Actually, he's a damn sight better than any o...  
5        "Cigarette smoking does not cause cancer." -Ce...  
13       "Environmentalists" Say Fracking is Evil 53-58...  
14       "Environmentalists" Say Fracking is Evil SAPER...  
15       "Environmentalists" Say Fracking is Evil This ...  
                                                       ocr  \
post_id                                                      
1        [(!! 

#IMPLEMENTATION


In [19]:
# Combine post text with fact claim for positive pairs
positive_pairs = []
for _, row in filtered_mapping.iterrows():
    post_text = df_monolingual_train_posts.loc[row['post_id'], 'text']
    fact_claim = filtered_facts.loc[row['fact_check_id'], 'claim']
    positive_pairs.append((post_text, fact_claim, 1))  # 1 = matched pair

# Create negative pairs
all_fact_ids = set(filtered_facts.index)
negative_pairs = []
for _, row in filtered_mapping.iterrows():
    post_text = df_monolingual_train_posts.loc[row['post_id'], 'text']
    # Randomly pick a fact_check_id not in mapping
    non_matching_facts = list(all_fact_ids - set([row['fact_check_id']]))
    if non_matching_facts:
        random_fact_id = random.choice(non_matching_facts)
        fact_claim = filtered_facts.loc[random_fact_id, 'claim']
        negative_pairs.append((post_text, fact_claim, 0))  # 0 = non-matched

# Combine positive and negative pairs
final_data = positive_pairs + negative_pairs
random.shuffle(final_data)


In [20]:
class FactCheckDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        post_text, fact_claim, label = self.data[idx]
        inputs = self.tokenizer(
            post_text,
            fact_claim,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "label": torch.tensor(label, dtype=torch.float)
        }

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")

# Create dataset and DataLoader
dataset = FactCheckDataset(final_data, tokenizer)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [21]:
class FactCheckModel(torch.nn.Module):
    def __init__(self, model_name="google-bert/bert-base-multilingual-cased"):
        super(FactCheckModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # CLS token embedding
        logits = self.classifier(cls_output)
        return logits

# Initialize model
model = FactCheckModel()


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [22]:
# Optimizer and Loss
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.BCEWithLogitsLoss()

# Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask).squeeze()
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_loader):.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/3: 100%|██████████| 681/681 [16:27<00:00,  1.45s/it]


Epoch 1, Loss: 0.2752


Epoch 2/3: 100%|██████████| 681/681 [16:31<00:00,  1.46s/it]


Epoch 2, Loss: 0.1053


Epoch 3/3: 100%|██████████| 681/681 [16:30<00:00,  1.45s/it]

Epoch 3, Loss: 0.0609


In [25]:
import os

# Create a directory to save the model
save_dir = "saved_model"
os.makedirs(save_dir, exist_ok=True)

# Save model and tokenizer
model_save_path = os.path.join(save_dir, "model.pth")
tokenizer_save_path = os.path.join(save_dir, "tokenizer")

torch.save(model.state_dict(), model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved at {model_save_path}")
print(f"Tokenizer saved at {tokenizer_save_path}")

Model saved at saved_model/model.pth
Tokenizer saved at saved_model/tokenizer


In [ ]:
# # Load the tokenizer
# tokenizer_path = "tokenizer"  # Adjust to where tokenizer is uploaded
# tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# # Load the model
# model_path = "model.pth"  # Adjust to where model is uploaded
# model = FactCheckModel()
# model.load_state_dict(torch.load(model_path))
# model.eval()
# model.to(device)  # Move the model to the appropriate device


In [23]:
from torch.nn.functional import cosine_similarity

# Generate embeddings for posts
model.eval()
post_embeddings = {}
with torch.no_grad():
    for idx, row in df_monolingual_train_posts.iterrows():
        inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=512).to(device)
        embedding = model.bert(**inputs).last_hidden_state[:, 0, :].squeeze()
        post_embeddings[idx] = embedding.cpu()

# Generate embeddings for facts
fact_embeddings = {}
with torch.no_grad():
    for idx, row in filtered_facts.iterrows():
        inputs = tokenizer(row['claim'], return_tensors="pt", truncation=True, max_length=512).to(device)
        embedding = model.bert(**inputs).last_hidden_state[:, 0, :].squeeze()
        fact_embeddings[idx] = embedding.cpu()

# Compute cosine similarity
similarity_results = {}
for post_id, post_emb in post_embeddings.items():
    scores = {fact_id: cosine_similarity(post_emb.unsqueeze(0), fact_emb.unsqueeze(0)).item()
              for fact_id, fact_emb in fact_embeddings.items()}
    similarity_results[post_id] = sorted(scores.items(), key=lambda x: x[1], reverse=True)



In [33]:
# Generate embeddings for development posts (df_mdp)
import pickle

df_mdp_embeddings = {}
with torch.no_grad():
    for idx, row in df_mdp.iterrows():
        inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=512).to(device)
        embedding = model.bert(**inputs).last_hidden_state[:, 0, :].squeeze()
        df_mdp_embeddings[idx] = embedding.cpu()

# Save df_mdp embeddings to a file
with open("df_mdp_embeddings.pkl", "wb") as f:
    pickle.dump(df_mdp_embeddings, f)

print("Development post embeddings saved!")


Development post embeddings saved!


In [34]:
# Save fact embeddings to a file
with open("fact_embeddings.pkl", "wb") as f:
    pickle.dump(fact_embeddings, f)

print("Fact embeddings saved!")

Fact embeddings saved!


In [35]:
from torch.nn.functional import cosine_similarity

# Compute cosine similarity for df_mdp embeddings and fact embeddings
similarity_results = {}
for post_id, post_emb in df_mdp_embeddings.items():
    scores = {fact_id: cosine_similarity(post_emb.unsqueeze(0), fact_emb.unsqueeze(0)).item()
              for fact_id, fact_emb in fact_embeddings.items()}
    similarity_results[post_id] = sorted(scores.items(), key=lambda x: x[1], reverse=True)

# # Example: Save top-N predictions for each post
# top_n = 5  # Adjust based on your requirements
# predictions = {
#     post_id: [fact_id for fact_id, _ in similarity_results[post_id][:top_n]]
#     for post_id in similarity_results.keys()
# }

# Save predictions to a file
with open("predictions.pkl", "wb") as f:
    pickle.dump(predictions, f)

print("Predictions saved!")


Predictions saved!


In [28]:
len(fact_embeddings)

3347

In [29]:
len(similarity_results)

4351

In [30]:
len(post_embeddings)

4351

In [23]:
def evaluate_model(mapping_df, similarity_df):
    correct = 0
    total = len(mapping_df)
    for _, row in mapping_df.iterrows():
        post_id = row['post_id']
        true_fact_id = row['fact_check_id']
        if true_fact_id in similarity_df.loc[post_id].tolist():
            correct += 1
    return correct / total

accuracy = evaluate_model(filtered_mapping, similarity_df)
print(f"Accuracy: {accuracy:.2%}")


In [24]:
# # Load fact embeddings
# with open("fact_embeddings.pkl", "rb") as f:
#     fact_embeddings = pickle.load(f)

# # Load development post embeddings
# with open("df_mdp_embeddings.pkl", "rb") as f:
#     df_mdp_embeddings = pickle.load(f)

# # Load predictions
# with open("predictions.pkl", "rb") as f:
#     predictions = pickle.load(f)

# print("Embeddings and predictions loaded!")

NameError: name 'fact_check_embeddings' is not defined

In [ ]:
# df_top_n_cosine

In [31]:
# Convert Series into the required dictionary format
submission_dict = {str(post_id): top_n_facts[:10] for post_id, top_n_facts in similarity_df.items()}

# Define the filename for the monolingual task
output_filename = "monolingual_1.json"

# Write the dictionary to a JSON file
with open(output_filename, 'w') as json_file:
    json.dump(similarity_results, json_file, indent=4)

print(f"Submission saved to {output_filename}")


Submission saved to monolingual_1.json


In [36]:
import json

# Format similarity results to desired JSON structure
top_n = 10
formatted_results = {
    post_id: [fact_id for fact_id, _ in similarity_results[post_id][:top_n]]
    for post_id in similarity_results.keys()
}

# Save to a JSON file
with open("similarity_results.json", "w") as json_file:
    json.dump(formatted_results, json_file, indent=4)

print("Similarity results formatted and saved to 'similarity_results.json'!")


Similarity results formatted and saved to 'similarity_results.json'!
